In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os,sys,scipy
import errno
import string
from nltk.corpus import reuters 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
import csv
from sklearn import svm
import warnings
import random
from statistics import mode
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
random.seed(0)
warnings.filterwarnings('ignore')
def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels
def loadTimeSeriesDatasets(directory, dataset):
    f=directory+"\\"+dataset+"\\"+dataset+'_TEST.arff'
    data = arff.loadarff(f)
    df = pd.DataFrame(data[0])
    test_labels=df['target']
    test_data= np.array(df.loc[:, df.columns != 'class_atr'])
    f=directory+"\\"+dataset+"\\"+dataset+'_TRAIN.arff'
    data = arff.loadarff(f)
    df = pd.DataFrame(data[0])
    train_labels=df['target']
    train_data= np.array(df.loc[:, df.columns != 'class_atr'])
    return train_data, train_labels, test_data, test_labels

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return 1-isc
def most_common(lst):
    return(mode(lst))


def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=list(range(1,26)),train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["k",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=train_time+test_time
    for i in range(0,len(Arr)):
        print(i,len(Arr))
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)
def ExNRule(xTrain, yTrain,xTest,k):
   
    B=500
    n, p = xTrain.shape
    y_pred=np.zeros((len(xTest),B))
    for b in range(B):
        # Bootstrap with p_subset features
        p_dash= int(math.sqrt(p))
        indices = np.random.choice(np.arange(p), size=p_dash, replace=False)
        xTrain_bs = xTrain[:,indices]
        xTest_bs = xTest[:,indices]
        y=np.zeros((len(xTest),k))
        x_subset=xTest_bs

        for i in range(k):
            # Nearest training observation in the bootstrap sample
            knn = NearestNeighbors(n_neighbors=1)
            knn.fit(xTrain_bs,yTrain)
            nearest_index = knn.kneighbors(x_subset, return_distance=False)
            x_subset = xTrain_bs[nearest_index]
            sh=x_subset.shape
            x_subset=x_subset.reshape(sh[0],sh[2])
            y[:,i] = yTrain[nearest_index].reshape(-1,)
        y_pred[:,b]= scipy.stats.mode(y.T)[0]

    pred = scipy.stats.mode(y_pred.T)[0]
    return pred
def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    averageMeanPrecison=[]
    kList=list(range(1,26))
    func=globals()[metric]
    train_time=[]
    test_time=[]
    maxval=np.max(kList)
    var=np.zeros(train_data.shape[1])
    class_dist=[]
    label_dist=[]
    ##########Main piece of code############
    # split training data according to class distribution
    for k in kList:
        time1=time.time()
        time2=time.time()
        pred=ExNRule(train_data,train_labels,test_data,k)
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
    class_dict={}
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)

        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)

    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
directory = "Time Series Datasets"
datasets = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
print(datasets)

 
for dataset in datasets:
    print(dataset)
    train_data,train_labels, test_data,test_labels=loadTimeSeriesDatasets("Time Series Datasets\\", dataset)
    labels=list(train_labels).copy()
    arr=list(train_data).copy()
    labels.extend(test_labels)
    arr.extend(test_data)
    labels=np.array(labels)
    arr=np.array(arr)
    le = preprocessing.LabelEncoder().fit(labels)
    labels=le.transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    classes=list(set(labels))
    global w
    w=0
    global Weights
    print("arr",arr.shape)
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "ExRule-KNN\\Time Series 5-Splits"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_ExRule-KNN-5Splits.txt'
        tables = open(fname, 'wb')
        k_splits=5
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        split_list=[]
        X = arr.copy()
        y = labels.copy()
        for i in range(0,5):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            split_list.append([X_train, X_test, y_train, y_test])
            X = X_train.copy()
            y = y_train.copy()
        for i in range(0,5):
            xTrain, xTest = split_list[i][0],split_list[i][1]
            yTrain, yTest = split_list[i][2],split_list[i][3]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=list(range(1,26))
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





[nltk_data] Downloading package reuters to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['Car', 'Fish', 'GunPoint', 'Ham', 'Haptics', 'Herring', 'InlineSkate', 'Meat', 'Plane', 'Strawberry', 'Trace', 'Yoga']
Car
arr (120, 578)
###############
{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+-------------------+-------------------+
|    k     | Accuracy | Train Time |     Test time     |     Total Time    |
+----------+----------+------------+-------------------+-------------------+
|    1     |  0.875   |  0:00:00   |   0:00:12.778867  |      0:00:00      |
|    2     |  0.9167  |  0:00:00   |   0:00:30.264504  |      0:00:00      |
|    3     |  0.9167  |  0:00:00   |   0:00:43.192151  |      0:00:00      |
|    4     |  0.875   |  0:00:00   |   0:11:31.075250  |      0:00:00      |
|    5     |  0.875   |  0:00:00   |   0:14:59.746377  |      0:00:00      |
|    6     |  0.9167  |  0:00:

{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+--------------------+--------------------+
|    k     | Accuracy | Train Time |     Test time      |     Total Time     |
+----------+----------+------------+--------------------+--------------------+
|    1     |   0.85   |  0:00:00   |   0:00:07.954793   |      0:00:00       |
|    2     |   0.85   |  0:00:00   |   0:00:11.860123   |      0:00:00       |
|    3     |   0.8    |  0:00:00   |   0:00:16.088639   |      0:00:00       |
|    4     |   0.8    |  0:00:00   |   0:00:23.204220   |      0:00:00       |
|    5     |   0.9    |  0:00:00   |   0:00:25.686764   |      0:00:00       |
|    6     |   0.9    |  0:00:00   |   0:00:29.545763   |      0:00:00       |
|    7     |   0.9    |  0:00:00   |   0:00:40.740254   |      0:00:00       |
|    8     

{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+--------------------+-------------------+
|    k     | Accuracy | Train Time |     Test time      |     Total Time    |
+----------+----------+------------+--------------------+-------------------+
|    1     |  0.6875  |  0:00:00   |   0:00:02.343203   |      0:00:00      |
|    2     |  0.6875  |  0:00:00   |   0:00:03.530424   |      0:00:00      |
|    3     |  0.6875  |  0:00:00   |   0:00:04.998832   |      0:00:00      |
|    4     |  0.6875  |  0:00:00   |   0:00:06.186052   |      0:00:00      |
|    5     |  0.6875  |  0:00:00   |   0:00:07.810674   |      0:00:00      |
|    6     |  0.6875  |  0:00:00   |   0:00:09.044760   |      0:00:00      |
|    7     |  0.6875  |  0:00:00   |   0:00:10.585627   |      0:00:00      |
|    8     |  0.6875 

{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+--------------------+--------------------+
|    k     | Accuracy | Train Time |     Test time      |     Total Time     |
+----------+----------+------------+--------------------+--------------------+
|    1     |  0.8333  |  0:00:00   |   0:00:02.405689   |      0:00:00       |
|    2     |  0.8333  |  0:00:00   |   0:00:03.905340   |      0:00:00       |
|    3     |  0.8333  |  0:00:00   |   0:00:05.417417   |      0:00:00       |
|    4     |  0.8333  |  0:00:00   |   0:00:07.202568   |      0:00:00       |
|    5     |  0.8333  |  0:00:00   |   0:00:08.517007   |      0:00:00       |
|    6     |  0.8333  |  0:00:00   |   0:00:10.141566   |      0:00:00       |
|    7     |  0.8333  |  0:00:00   |   0:00:11.704332   |      0:00:00       |
|    8     

...

arr (3300, 427)
###############
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+-------------------+-------------------+
|    k     | Accuracy | Train Time |     Test time     |     Total Time    |
+----------+----------+------------+-------------------+-------------------+
|    1     |  0.9364  |  0:00:00   |   0:00:31.541523  |      0:00:00      |
|    2     |  0.9379  |  0:00:00   |   0:00:39.242538  |      0:00:00      |
|    3     |  0.9409  |  0:00:00   |   0:00:46.780560  |      0:00:00      |
|    4     |  0.9409  |  0:00:00   |   0:00:53.977028  |      0:00:00      |
|    5     |  0.9439  |  0:00:00   |   0:01:01.534187  |      0:00:00      |
|    6     |  0.9394  |  0:00:00   |   0:01:08.881566  |      0:00:00      |
|    7     |  0.9439  |  0:00:00   |   0:01:16.130854  |      0:00:00      |
|    8     

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+-------------------+-------------------+
|    k     | Accuracy | Train Time |     Test time     |     Total Time    |
+----------+----------+------------+-------------------+-------------------+
|    1     |  0.9432  |  0:00:00   |   0:00:26.394223  |      0:00:00      |
|    2     |  0.9432  |  0:00:00   |   0:00:32.271800  |      0:00:00      |
|    3     |  0.9413  |  0:00:00   |   0:00:38.284660  |      0:00:00      |
|    4     |  0.9432  |  0:00:00   |   0:00:46.208595  |      0:00:00      |
|    5     |  0.947   |  0:00:00   |   0:00:50.750385  |      0:00:00      |
|    6     |  0.9394  |  0:00:00   |   0:00:58.930935  |      0:00:00      |
|    7     |  0.9451  |  0:00:00   |   0:01:03.814181  |      0:00:00      |
|    8     |  0.9394  |  0:00:00   |   0:01

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+-------------------+-------------------+
|    k     | Accuracy | Train Time |     Test time     |     Total Time    |
+----------+----------+------------+-------------------+-------------------+
|    1     |  0.9598  |  0:00:00   |   0:00:18.345990  |      0:00:00      |
|    2     |  0.9645  |  0:00:00   |   0:00:22.238358  |      0:00:00      |
|    3     |  0.9622  |  0:00:00   |   0:00:27.190533  |      0:00:00      |
|    4     |  0.9645  |  0:00:00   |   0:00:33.438191  |      0:00:00      |
|    5     |  0.9622  |  0:00:00   |   0:00:37.032299  |      0:00:00      |
|    6     |  0.9622  |  0:00:00   |   0:00:41.065073  |      0:00:00      |
|    7     |  0.9574  |  0:00:00   |   0:00:47.821366  |      0:00:00      |
|    8     |  0.9598  |  0:00:00   |   0:00

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+--------------------+--------------------+
|    k     | Accuracy | Train Time |     Test time      |     Total Time     |
+----------+----------+------------+--------------------+--------------------+
|    1     |  0.9083  |  0:00:00   |   0:00:13.421956   |      0:00:00       |
|    2     |  0.9112  |  0:00:00   |   0:00:18.129908   |      0:00:00       |
|    3     |  0.9112  |  0:00:00   |   0:00:21.505634   |      0:00:00       |
|    4     |  0.9083  |  0:00:00   |   0:00:25.566405   |      0:00:00       |
|    5     |  0.9083  |  0:00:00   |   0:00:29.422905   |      0:00:00       |
|    6     |  0.9053  |  0:00:00   |   0:00:33.342695   |      0:00:00       |
|    7     |  0.9053  |  0:00:00   |   0:00:37.142807   |      0:00:00       |
|    8     |  0.9083  |

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+------------+------------------+------------------+
|    k     | Accuracy | Train Time |    Test time     |    Total Time    |
+----------+----------+------------+------------------+------------------+
|    1     |  0.9151  |  0:00:00   |  0:00:12.090892  |     0:00:00      |
|    2     |  0.9151  |  0:00:00   |  0:00:14.381286  |     0:00:00      |
|    3     |  0.9151  |  0:00:00   |  0:00:18.005567  |     0:00:00      |
|    4     |  0.9114  |  0:00:00   |  0:00:20.801860  |     0:00:00      |
|    5     |  0.9114  |  0:00:00   |  0:00:26.356580  |     0:00:00      |
|    6     |  0.9225  |  0:00:00   |  0:00:30.504297  |     0:00:00      |
|    7     |  0.9114  |  0:00:00   |  0:00:31.885754  |     0:00:00      |
|    8     |  0.9114  |  0:00:00   |  0:00:34.852615  |     0:0

In [2]:
test=[1,2,3,4]
print(test[0:-1])

[1, 2, 3]


In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y_scores = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
roc_auc_score(y_true, y_scores)

0.5